In [1]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [3]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [4]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [5]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [6]:
ct=pd.read_csv('a0529-a0611_UK/a0530.csv',encoding='ISO-8859-1')

In [7]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [8]:
###
#ct=pd.read_csv('a0601.csv')

In [9]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,575867902590661_1494105610766881,Facebook,2020-05-30 23:59:56,2021-07-19 07:35:46,photo,NaN,NaN,Tomorrow's Sunday National front page: Global ...,OPEN JOINT LETTER: The Lord Advocate Mr Walter...,1.0,https://www.facebook.com/thenationalnewspapers...,https://www.facebook.com/575867902590661/posts...,0,-2.951220,1.0,28,8,1,4,0,0,0,0,0,0,55,12,6,11,2,23,2,10,0,2,12754183,Aye Aberdeen,AyeAberdeen,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,"Since Brexit, Scotland has been repeatedly ign...",False,2016-07-07 17:40:55,en,0,12754183|1494105610766881,"{'Edinburgh': 1, 'Scotland': 6, 'UK': 1, 'US':...",{},{}
1,1,174766919255146_271144980752352,Facebook,2020-05-30 23:59:52,2021-07-01 21:21:40,live_video_complete,NaN,NaN,Tomorrow's Sunday National front page: Global ...,George Floyd Memorial Go Fund Me: https://www....,4.0,https://www.facebook.com/SweetH20MusicHall/vid...,https://www.facebook.com/174766919255146/posts...,48081,1.500000,2.0,33,13,20,11,0,0,0,1,0,1,11,9,19,9,1,1,1,1,0,2,403510,Sweetwater Music Hall,SweetH20MusicHall,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Sweetwater Music Hall is a landmark live music...,True,2011-07-10 22:02:00,en,0,403510|271144980752352,{},{},{}
2,2,104129837848_10157318590807849,Facebook,2020-05-30 23:59:50,2021-08-31 10:53:12,link,Cities brace for increasing unrest as anger ov...,kgun9.com,"Protesters set police cars ablaze, smashed bus...","Protesters set police cars ablaze, smashed bus...",1.0,https://www.kgun9.com/news/national/cities-bra...,https://www.facebook.com/104129837848/posts/10...,200968,-1.194444,1.0,19,3,80,0,1,2,13,26,0,1,54,14,45,13,6,12,10,18,0,3,20324,KGUN 9 On Your Side,KGUN9OnYourSide,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Visit us at kgun9.com\nFollow us on IG https:/...,True,2009-06-11 22:36:58,en,0,20324|10157318590807849,"{'Atlanta': 1, 'Los Angeles': 1}","{'Atlanta': 1, 'Los Angeles': 1}",{}
3,3,346979175820702_883214425530505,Facebook,2020-05-30 23:59:50,2021-06-18 19:51:53,link,Hundreds of thousands stage multiracial demons...,wsws.org,Mass demonstrations of workers and youth of al...,The police are not a cross section of American...,1.0,https://www.wsws.org/en/articles/2020/05/30/pe...,https://www.facebook.com/346979175820702/posts...,0,-0.750000,1.0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,14859738,Victoria University of Wellington Students for...,vuwiysse,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Associated with IYSSE and the Social Equality ...,False,2018-03-21 22:42:39,en,0,14859738|883214425530505,{},{},{}
4,4,624208120942702_3471299859566833,Facebook,2020-05-30 23:59:50,2021-05-27 00:13:54,link,Anglican Church in North America,anglicanchurch.net,Anglican Church in North America,We are grateful for this letter from ACNA bish...,2.0,http://www.anglicanchurch.net/?%2Fmain%2Fpage%...,https://www.facebook.com/624208120942702/posts...,804,0.684211,1.0,11,0,0,2,0,0,0,0,0,0,4,1,1,2,0,0,0,0,0,0,3518248,Immanuel Anglican Church of Chicago,chicagoanglican,https://scontent-den4-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Immanuel Anglican Church is a congregation in ...,False,2013-05-14 02:31:46,en,0,3518248|3471299859566833,{},{},{}
...,...,..

In [10]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [11]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [12]:
###
ct_ner = ct

In [13]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Edinburgh': 1, 'Scotland': 6, 'UK': 1, 'US':...",{},{}
1,{},{},{}
2,"{'Atlanta': 1, 'Los Angeles': 1}","{'Atlanta': 1, 'Los Angeles': 1}",{}
3,{},{},{}
4,{},{},{}


In [14]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [15]:
ct_ner['NER-msg']

0        [US, UK, Scotland, Edinburgh, Convenor, Iraq]
1                                                   []
2                               [Atlanta, Los Angeles]
3                                                   []
4                                                   []
                             ...                      
35362                                        [Atlanta]
35363                                        [Atlanta]
35364                     [NYC, Brooklyn, Minneapolis]
35365                             [Deep Ellum, Dallas]
35366                                             [us]
Name: NER-msg, Length: 35367, dtype: object

In [16]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [17]:
ct_ner['NER']

0        [US, UK, Scotland, Edinburgh, Convenor, Iraq]
1                                                   []
2         [Atlanta, Los Angeles, Atlanta, Los Angeles]
3                                                   []
4                                                   []
                             ...                      
35362                               [Atlanta, Atlanta]
35363                               [Atlanta, Atlanta]
35364            [NYC, Brooklyn, Minneapolis, Atlanta]
35365                    [Deep Ellum, Dallas, Atlanta]
35366                                             [us]
Name: NER, Length: 35367, dtype: object

In [18]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [20]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,575867902590661_1494105610766881,Facebook,2020-05-30 23:59:56,2021-07-19 07:35:46,photo,NaN,NaN,Tomorrow's Sunday National front page: Global ...,OPEN JOINT LETTER: The Lord Advocate Mr Walter...,1.0,https://www.facebook.com/thenationalnewspapers...,https://www.facebook.com/575867902590661/posts...,0,-2.951220,1.0,28,8,1,4,0,0,0,0,0,0,55,12,6,11,2,23,2,10,0,2,12754183,Aye Aberdeen,AyeAberdeen,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,"Since Brexit, Scotland has been repeatedly ign...",False,2016-07-07 17:40:55,en,0,12754183|1494105610766881,"[US, UK, Scotland, Edinburgh, Convenor, Iraq]",[],[],"[us, uk, scotland, edinburgh, convenor, iraq]"
1,1,174766919255146_271144980752352,Facebook,2020-05-30 23:59:52,2021-07-01 21:21:40,live_video_complete,NaN,NaN,Tomorrow's Sunday National front page: Global ...,George Floyd Memorial Go Fund Me: https://www....,4.0,https://www.facebook.com/SweetH20MusicHall/vid...,https://www.facebook.com/174766919255146/posts...,48081,1.500000,2.0,33,13,20,11,0,0,0,1,0,1,11,9,19,9,1,1,1,1,0,2,403510,Sweetwater Music Hall,SweetH20MusicHall,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Sweetwater Music Hall is a landmark live music...,True,2011-07-10 22:02:00,en,0,403510|271144980752352,[],[],[],[]
2,2,104129837848_10157318590807849,Facebook,2020-05-30 23:59:50,2021-08-31 10:53:12,link,Cities brace for increasing unrest as anger ov...,kgun9.com,"Protesters set police cars ablaze, smashed bus...","Protesters set police cars ablaze, smashed bus...",1.0,https://www.kgun9.com/news/national/cities-bra...,https://www.facebook.com/104129837848/posts/10...,200968,-1.194444,1.0,19,3,80,0,1,2,13,26,0,1,54,14,45,13,6,12,10,18,0,3,20324,KGUN 9 On Your Side,KGUN9OnYourSide,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Visit us at kgun9.com\nFollow us on IG https:/...,True,2009-06-11 22:36:58,en,0,20324|10157318590807849,"[Atlanta, Los Angeles]","[Atlanta, Los Angeles]",[],"[atlanta, los angeles]"
3,3,346979175820702_883214425530505,Facebook,2020-05-30 23:59:50,2021-06-18 19:51:53,link,Hundreds of thousands stage multiracial demons...,wsws.org,Mass demonstrations of workers and youth of al...,The police are not a cross section of American...,1.0,https://www.wsws.org/en/articles/2020/05/30/pe...,https://www.facebook.com/346979175820702/posts...,0,-0.750000,1.0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,14859738,Victoria University of Wellington Students for...,vuwiysse,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Associated with IYSSE and the Social Equality ...,False,2018-03-21 22:42:39,en,0,14859738|883214425530505,[],[],[],[]
4,4,624208120942702_3471299859566833,Facebook,2020-05-30 23:59:50,2021-05-27 00:13:54,link,Anglican Church in North America,anglicanchurch.net,Anglican Church in North America,We are grateful for this letter from ACNA bish...,2.0,http://www.anglicanchurch.net/?%2Fmain%2Fpage%...,https://www.facebook.com/624208120942702/posts...,804,0.684211,1.0,11,0,0,2,0,0,0,0,0,0,4,1,1,2,0,0,0,0,0,0,3518248,Immanuel Anglican Church of Chicago,chicagoanglican,https://scontent-den4-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Immanuel Anglican Church is a congregation in ...,False,2013-05-14

In [21]:
ct_ner['NER-1']=ct_ner['NER']

In [22]:
ct_ner['NER-1'][1]

[]

In [23]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 5.4 MB/s 


In [24]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [25]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,575867902590661_1494105610766881,Facebook,2020-05-30 23:59:56,2021-07-19 07:35:46,photo,NaN,NaN,Tomorrow's Sunday National front page: Global ...,OPEN JOINT LETTER: The Lord Advocate Mr Walter...,1.0,https://www.facebook.com/thenationalnewspapers...,https://www.facebook.com/575867902590661/posts...,0,-2.951220,1.0,28,8,1,4,0,0,0,0,0,0,55,12,6,11,2,23,2,10,0,2,12754183,Aye Aberdeen,AyeAberdeen,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,"Since Brexit, Scotland has been repeatedly ign...",False,2016-07-07 17:40:55,en,0,12754183|1494105610766881,"[US, UK, Scotland, Edinburgh, Convenor, Iraq]",[],[],"[us, uk, scotland, edinburgh, convenor, iraq]","[us, uk, scotland, edinburgh, convenor, iraq]"
1,1,174766919255146_271144980752352,Facebook,2020-05-30 23:59:52,2021-07-01 21:21:40,live_video_complete,NaN,NaN,Tomorrow's Sunday National front page: Global ...,George Floyd Memorial Go Fund Me: https://www....,4.0,https://www.facebook.com/SweetH20MusicHall/vid...,https://www.facebook.com/174766919255146/posts...,48081,1.500000,2.0,33,13,20,11,0,0,0,1,0,1,11,9,19,9,1,1,1,1,0,2,403510,Sweetwater Music Hall,SweetH20MusicHall,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Sweetwater Music Hall is a landmark live music...,True,2011-07-10 22:02:00,en,0,403510|271144980752352,[],[],[],[],[]
2,2,104129837848_10157318590807849,Facebook,2020-05-30 23:59:50,2021-08-31 10:53:12,link,Cities brace for increasing unrest as anger ov...,kgun9.com,"Protesters set police cars ablaze, smashed bus...","Protesters set police cars ablaze, smashed bus...",1.0,https://www.kgun9.com/news/national/cities-bra...,https://www.facebook.com/104129837848/posts/10...,200968,-1.194444,1.0,19,3,80,0,1,2,13,26,0,1,54,14,45,13,6,12,10,18,0,3,20324,KGUN 9 On Your Side,KGUN9OnYourSide,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Visit us at kgun9.com\nFollow us on IG https:/...,True,2009-06-11 22:36:58,en,0,20324|10157318590807849,"[Atlanta, Los Angeles]","[Atlanta, Los Angeles]",[],"[atlanta, los angeles]","[atlanta, los angeles]"
3,3,346979175820702_883214425530505,Facebook,2020-05-30 23:59:50,2021-06-18 19:51:53,link,Hundreds of thousands stage multiracial demons...,wsws.org,Mass demonstrations of workers and youth of al...,The police are not a cross section of American...,1.0,https://www.wsws.org/en/articles/2020/05/30/pe...,https://www.facebook.com/346979175820702/posts...,0,-0.750000,1.0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,14859738,Victoria University of Wellington Students for...,vuwiysse,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Associated with IYSSE and the Social Equality ...,False,2018-03-21 22:42:39,en,0,14859738|883214425530505,[],[],[],[],[]
4,4,624208120942702_3471299859566833,Facebook,2020-05-30 23:59:50,2021-05-27 00:13:54,link,Anglican Church in North America,anglicanchurch.net,Anglican Church in North America,We are grateful for this letter from ACNA bish...,2.0,http://www.anglicanchurch.net/?%2Fmain%2Fpage%...,https://www.facebook.com/624208120942702/posts...,804,0.684211,1.0,11,0,0,2,0,0,0,0,0,0,4,1,1,2,0,0,0,0,0,0,3518248,Immanuel Anglican Church of Chicago,chicagoanglican,https://scontent-den4-1.xx.fbcdn.net/v/t31.181..

In [26]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [27]:
!ls

dataframe0529_post.csv	    dataframe_mapping_0604.csv
dataframe_mapping_0529.csv  dataframe_mapping_0605.csv
dataframe_mapping_0530.csv  dataframe_mapping_0606.csv
dataframe_mapping_0531.csv  dataframe_mapping_0607.csv
dataframe_mapping_0601.csv  dataframe_mapping_0608.csv
dataframe_mapping_0602.csv  dataframe_mapping_0609.csv
dataframe_mapping_0603.csv  dataframe_mapping_0610.csv


In [28]:
ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [29]:
len(ct_mapping['City'])

2945

In [30]:
len(ct_ner['NER-1'])

35367

In [31]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [32]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,575867902590661_1494105610766881,Facebook,2020-05-30 23:59:56,2021-07-19 07:35:46,photo,NaN,NaN,Tomorrow's Sunday National front page: Global ...,OPEN JOINT LETTER: The Lord Advocate Mr Walter...,1.0,https://www.facebook.com/thenationalnewspapers...,https://www.facebook.com/575867902590661/posts...,0,-2.951220,1.0,28,8,1,4,0,0,0,0,0,0,55,12,6,11,2,23,2,10,0,2,12754183,Aye Aberdeen,AyeAberdeen,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,"Since Brexit, Scotland has been repeatedly ign...",False,2016-07-07 17:40:55,en,0,12754183|1494105610766881,"[US, UK, Scotland, Edinburgh, Convenor, Iraq]",[],[],"[us, uk, scotland, edinburgh, convenor, iraq]","[United States, United Kingdom, Scotland, Unit..."
1,1,174766919255146_271144980752352,Facebook,2020-05-30 23:59:52,2021-07-01 21:21:40,live_video_complete,NaN,NaN,Tomorrow's Sunday National front page: Global ...,George Floyd Memorial Go Fund Me: https://www....,4.0,https://www.facebook.com/SweetH20MusicHall/vid...,https://www.facebook.com/174766919255146/posts...,48081,1.500000,2.0,33,13,20,11,0,0,0,1,0,1,11,9,19,9,1,1,1,1,0,2,403510,Sweetwater Music Hall,SweetH20MusicHall,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Sweetwater Music Hall is a landmark live music...,True,2011-07-10 22:02:00,en,0,403510|271144980752352,[],[],[],[],[]
2,2,104129837848_10157318590807849,Facebook,2020-05-30 23:59:50,2021-08-31 10:53:12,link,Cities brace for increasing unrest as anger ov...,kgun9.com,"Protesters set police cars ablaze, smashed bus...","Protesters set police cars ablaze, smashed bus...",1.0,https://www.kgun9.com/news/national/cities-bra...,https://www.facebook.com/104129837848/posts/10...,200968,-1.194444,1.0,19,3,80,0,1,2,13,26,0,1,54,14,45,13,6,12,10,18,0,3,20324,KGUN 9 On Your Side,KGUN9OnYourSide,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Visit us at kgun9.com\nFollow us on IG https:/...,True,2009-06-11 22:36:58,en,0,20324|10157318590807849,"[Atlanta, Los Angeles]","[Atlanta, Los Angeles]",[],"[atlanta, los angeles]","[Atlanta, Fulton County, Georgia, 30303-3506, ..."
3,3,346979175820702_883214425530505,Facebook,2020-05-30 23:59:50,2021-06-18 19:51:53,link,Hundreds of thousands stage multiracial demons...,wsws.org,Mass demonstrations of workers and youth of al...,The police are not a cross section of American...,1.0,https://www.wsws.org/en/articles/2020/05/30/pe...,https://www.facebook.com/346979175820702/posts...,0,-0.750000,1.0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,14859738,Victoria University of Wellington Students for...,vuwiysse,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Associated with IYSSE and the Social Equality ...,False,2018-03-21 22:42:39,en,0,14859738|883214425530505,[],[],[],[],[]
4,4,624208120942702_3471299859566833,Facebook,2020-05-30 23:59:50,2021-05-27 00:13:54,link,Anglican Church in North America,anglicanchurch.net,Anglican Church in North America,We are grateful for this letter from ACNA bish...,2.0,http://www.anglicanchurch.net/?%2Fmain%2Fpage%...,https://www.facebook.com/624208120942702/posts...,804,0.684211,1.0,11,0,0,2,0,0,0,0,0,0,4,1,1,2,0,0,0,0,0,0,3518248,Immanuel Anglican Church of Chicago,chicagoanglican,https://scontent-

In [33]:
ct_ner['NER-1'][12]

['Minneapolis, Hennepin County, Minnesota, 55415:55454, United States']

In [34]:
ct_ner.to_csv('dataframe0530_post.csv')